# Coupang

In [30]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()

#Input URL
url = "https://www.coupang.com/vp/products/7735720128?itemId=20799094145&vendorItemId=87868210326&pickType=COU_PICK&q=%EB%B0%98%ED%8C%94&itemsCount=36&searchId=0900a565c543450fb85923e270b43539&rank=1&isAddedCart="
driver.get(url)

#js load 될 떄 까지 delay
script = """return document.readyState === 'complete';"""
while not driver.execute_script(script):
    pass

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

try:
    #product name
    product_name = soup.find_all('h1', class_='prod-buy-header__title')
    product_price = soup.find_all('span', class_='total-price')
    
    #get option each type
    option_type1_name = soup.find_all('div', class_='option-table-list__option-name')
    option_type1_price = soup.find_all('div', class_='option-table-list__option-price')

    option_type2_name = soup.find_all('div', class_='prod-option__dropdown-item-title')
    option_type2_price = soup.find_all('div', class_="prod-option__dropdown-item-price")

    option_type3_name = soup.find_all('li', class_='Dropdown-Select__Dropdown__Item')
    option_type3_type = soup.find_all('span', class_='Text-Select__Item__Text')

except Exception as e:
    print(f"Error: {e}")

print("Product Name :")
for i in range(len(product_name)):
    print(f"{product_name[i].get_text(strip=True)} {product_price[i].get_text(strip=True)}")

print("Option List :")

TYPE = 0
if(len(option_type1_name) > 0):
    TYPE = 1
elif(len(option_type2_name) > 0):
    TYPE = 2
elif(len(option_type3_name) > 0):
    TYPE = 3

if(TYPE == 1):
    for i in range(len(option_type1_name)):
        print(f"{option_type1_name[i].get_text(strip=True)} {option_type1_price[i].get_text(strip=True)}")
elif(TYPE == 2):
    for i in range(len(option_type2_name)):
        print(f"{option_type2_name[i].get_text(strip=True)} {option_type2_price[i].get_text(strip=True)}")
elif(TYPE == 3):
    for i in range(len(option_type3_name)):
        print(f"{option_type3_name[i].get_text(strip=True)}")
    for i in range(len(option_type3_type)):
        print(f"{option_type3_type[i].get_text(strip=True)}")

driver.quit()

Product Name :
캐럿 기본 라운드 반팔 티셔츠 3p 12,630원
Option List :
M (95)
L (100)
XL (105)
2XL (110)


# Gmarket (옵션 구현 진행중)

In [23]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import json

driver = webdriver.Chrome()

#Input URL
url = "https://item.gmarket.co.kr/Item?goodscode=2522809147"
driver.get(url)

#js load 될 떄 까지 delay
script = """return document.readyState === 'complete';"""
while not driver.execute_script(script):
    pass

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

try:
    #product name
    product_title = soup.find('title')

    #product options
    product_name = soup.find_all('span', class_='item_tit')

    #options js
    scripts = soup.find_all('script')

except Exception as e:
    print(f"Error: {e}")

print("Product Name :")
print(f"{product_title.get_text(strip=True)}")

print("Product Option :")
#options
for i in range(len(product_name)):
    print(f"{product_name[i].get_text(strip=True)}")

for script in scripts:
    if script.string:
        #split crawling js to 1 line
        lines = script.string.splitlines()
        for line in lines:
            #option이 들어가있는 내용이면, 가져옴
            if "GmktItem.OptionParamCoreAbove.combOptionObj" in line:
                js = line
                break
#crawling result -> json

#js_result = re.find(r"GmktItem\.OptionParamCoreAbove\.combOptionObj\s*=\s*(\{.*?\}(?:,\s*\{.*?\})*);", js, re.DOTALL)
js_result = driver.execute_script("return GmktItem.OptionParamCoreAbove.combOptionObj;")

option_json = json.dumps(js_result, indent=4)  # 보기 좋게 포맷하기 위해 indent 사용

try:
    data = json.loads(option_json)
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)
    data = None

if(data):
    for key, value in data.items():
        if value and isinstance(value, dict):
            option_name = value.get('OptionName', '')
            print(f"Option Name: {option_name}")
            option_values = value.get('OptionValues', [])
            
            for option in option_values:
                option_no = option.get('OptionNo', '')
                option_value1 = option.get('OptionValue1', '')
                option_value2 = option.get('OptionValue2', '')
                option_value3 = option.get('OptionValue3', '')

                if(option_value1):
                    print(f"   Option1 : {option_value1}", end="")
                if(option_value2):
                    print(f"   Option2 : {option_value2}", end="")
                if(option_value3):
                    print(f"   Option3 : {option_value3}", end="")
                print()
            print("-" * 50)
else:
    print("This product has no option.")
driver.quit()


Product Name :
G마켓 - 파머스픽 유명산지 수박 9~10kg미만
Product Option :
This product has no option.


# Auction 99% 완

In [10]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import json

driver = webdriver.Chrome()

#Input URL
url = "http://itempage3.auction.co.kr/DetailView.aspx?itemno=E303854392"
driver.get(url)

#js load 될 떄 까지 delay
script = """return document.readyState === 'complete';"""
while not driver.execute_script(script):
    pass

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

try:
    #product name
    product_title = soup.find('title')
    product_name = soup.find_all('span', class_='item_tit')
    product_price = soup.find('span', class_ ="item_price")

    #option
    option_type1_name = soup.find_all('span', class_ = "option_prod")
    
    scripts = soup.find_all('script')

except Exception as e:
    print(f"Error: {e}")

print("Product Name :")
print(f"{product_title.get_text(strip=True)}")

print("Product Option :")
#options
for i in range(1, len(option_type1_name) // 2):
    print(f"{option_type1_name[i].get_text(strip=True)}")

for script in scripts:
    if script.string:
        #split crawling js to 1 line
        lines = script.string.splitlines()
        for line in lines:
            #option이 들어가있는 내용이면, 가져옴
            if "Request.ItemRequests=" in line:
                js = line
                break

#crawling result -> json
js_result = re.findall(r"Request\.ItemRequests\s*=\s*(\[\[.*?\]\]);", js, re.DOTALL)
if(len(js_result) > 0):
    option_json = js_result[0]

    try:
        data = json.loads(option_json)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        data = None

    option_list = {}
    if data:
        for outer_list in data:
            for item in outer_list:
                if 'OrderSectionName' in item:
                    
                    option_1 = item['OrderSectionName']
                    option_2 = item['OrderText']
                    option_3 = item['OrderText2']
                    option_4 = item['OrderText']

                    if(option_1 not in option_list):
                        option_list[option_1] = []
                    else:
                        option_list[option_1].append((option_2, option_3, option_4))

    for key, items in option_list.items():
        print(f"{key}:")
        for item in items:
            print(f"  - {item[0]}")
        print()
else:
    print("This product has no option.")

driver.quit()

Exception ignored in: <function Service.__del__ at 0x000001DC75C44430>
Traceback (most recent call last):
  File "c:\Users\Carly\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "c:\Users\Carly\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\Carly\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "c:\Users\Carly\anaconda3\lib\urllib\request.py", line 214, in urlopen
    return opener.open(url, data, timeout)
  File "c:\Users\Carly\anaconda3\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\Users\Carly\anaconda3\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\Users\Carly\anaconda3\lib\urllib\request.py", 

Product Name :
오리온 초코파이하우스 초코앤크림 408g - 옥션
Product Option :
This product has no option.


# 11ST

In [26]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import json

driver = webdriver.Chrome()

#Input URL
url = "https://www.11st.co.kr/products/2416794313?&trTypeCd=PW00&trCtgrNo=585021&checkCtlgPrd=true"
driver.get(url)

#js load 될 떄 까지 delay
script = """return document.readyState === 'complete';"""
while not driver.execute_script(script):
    pass


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

try:
    #product name
    product_title = soup.find('title')

    for div_tag in soup.find_all('div', class_='option_item_info cont'):
        strong_tags = div_tag.find_all('strong') 
        for strong_tag in strong_tags:
            print(strong_tag.text.strip())  

    scripts = soup.find_all('script')

except Exception as e:
    print(f"Error: {e}")

print("Product Name :")
print(f"{product_title.get_text(strip=True)}")


for script in scripts:
    if script.string:
        #split crawling js to 1 line
        lines = script.string.splitlines()
        for line in lines:
            print(line)
            print()

driver.quit()

HC303셔츠
HC307셔츠
HC277셔츠
HC273셔츠
보정 나시 2컬러 택1
일반핏 반팔 와이셔츠 6컬러 택1
HC278셔츠
HC274셔츠
HC260셔츠
데님 셔츠 2컬러 택1
스트라이프 셔츠 6컬러 택1
HC210셔츠
린넨 셔츠 8컬러 택1
HC217셔츠
린넨 7부 셔츠 3컬러 택1
HC286셔츠
HC250셔츠
HC367셔츠
HC245셔츠
하와이안셔츠 4종 택1
HC275셔츠
옥스포드 셔츠 3컬러 택1
하와이안셔츠 6종 택1
HC203셔츠
HC244셔츠
차이나 카라 셔츠 3컬러 택1
HC218셔츠
패턴 카라 셔츠 2컬러 택1
쿨링 7부 셔츠 5컬러 택1
HC232셔츠
HC370셔츠
HC252셔츠
HC212셔츠
HC262셔츠
HC207셔츠
HC268셔츠
드레스 셔츠 2컬러 택1
HC249셔츠
HC364셔츠
HC366셔츠
HC242셔츠
HC238셔츠
HC205셔츠
HC288셔츠
HC368셔츠
HC214셔츠
혼합 패턴 반팔 와이셔츠 13컬러 택1
스트라이프 반팔 와이셔츠 4컬러 택1
무지 슬림핏 반팔 와이셔츠 4컬러 택1
체크 반팔 와이셔츠 3컬러 택1
무지 반팔 와이셔츠 4컬러 택1
HC206셔츠
Product Name :
[95-140] 여름 남자 반팔 셔츠/빅사이즈/와이셔츠/린넨/남방/체크/오버핏/7부/스트라이프/하와이안/큰옷


    var rakeLog = (function(rakeLog){

        var empty = function() {};



        rakeLog.refresh = empty;

        rakeLog.pageView = empty;

        rakeLog.sendRakeLog = empty;

        rakeLog.setEnableLog = empty;

        rakeLog.scrollHandler = empty;

        rakeLog.getShuttle = function() {

            if ( LogClientSentinelShuttle ) {

  

# Naver Shopping

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import json

driver = webdriver.Chrome()

#Input URL
url = "https://www.11st.co.kr/products/2416794313?&trTypeCd=PW00&trCtgrNo=585021&checkCtlgPrd=true"
driver.get(url)

#js load 될 떄 까지 delay
script = """return document.readyState === 'complete';"""
while not driver.execute_script(script):
    pass


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

try:
    #product name
    product_title = soup.find('title')

    for div_tag in soup.find_all('div', class_='option_item_info cont'):
        strong_tags = div_tag.find_all('strong') 
        for strong_tag in strong_tags:
            print(strong_tag.text.strip())  

    scripts = soup.find_all('script')

except Exception as e:
    print(f"Error: {e}")

print("Product Name :")
print(f"{product_title.get_text(strip=True)}")


for script in scripts:
    if script.string:
        #split crawling js to 1 line
        lines = script.string.splitlines()
        for line in lines:
            print(line)
            print()

driver.quit()